In [15]:
## LIBRARIES AND CHROMEDRIVER STUFF - JUST RUN THIS CELL ONCE

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from IPython.display import display
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import os
import numpy as np

def get_chrome_options():
    
    chrome_options = Options()        
    chrome_options.add_argument('--headless=new')
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36")
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
    chrome_options.add_argument('--log-level=3')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--silent')
    chrome_options.add_argument('--disable-logging')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--no-sandbox')

    # Create service object with log output suppressed
    service = Service('C://chromedriver-win64//chromedriver.exe')
    service.creation_flags = 0x08000000  # Suppress console window

    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [13]:
topN = 40 ## GET THE PLAYERS/TEAMS INFO FOR THE TOP X HLTV TEAMS ## 
url = "https://www.hltv.org/ranking/teams/2025/february/10"

driver = get_chrome_options()

driver.get(url)

try:
    cookie_accept_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    )
    cookie_accept_button.click()
except Exception as e:
    print("Cookies notification not found:", e)

# Wait for the page to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "ranked-team"))
)

# Retrieve the HTML content of the page
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

# Find the first 50 ranked teams
teams = soup.find_all('div', class_='ranked-team standard-box', limit=topN)

team_data = []

for team in teams:
    nameTeam = team.find('span', class_='name').text
    pageTeam = f"https://www.hltv.org{team.find('a', class_='moreLink')['href'].replace(' ', '')}"
    allPlayers = team.find_all('td', class_='player-holder')
    
    if(len(allPlayers) == 4):
        team_data.append({
            'nameTeam': nameTeam,
            'pageTeam': pageTeam,
            'namePlayer': "missing_player",
            'countryPlayer': "n/a",
            'nickPlayer': "missing_player"
        })
    
    for player in allPlayers:
        
        namePlayer = player.find('img', class_='playerPicture')['title']
        countryPlayer = player.find('img', class_='gtSmartphone-only flag')['alt']
        nickPlayer = player.find('div', class_='nick').text.strip()
        
        team_data.append({
            'nameTeam': nameTeam,
            'pageTeam': pageTeam,
            'namePlayer': namePlayer,
            'countryPlayer': countryPlayer,
            'nickPlayer': nickPlayer
        })

driver.quit()

# Create a pandas DataFrame from the team_data list
df = pd.DataFrame(team_data)

pd.set_option('display.max_rows', None)
display(df)
# df.to_csv('hltv_teams.csv',encoding='utf-8-sig', index=False)

,nameTeam,pageTeam,namePlayer,countryPlayer,nickPlayer
0,Spirit,https://www.hltv.org/team/7020/spirit,Leonid 'chopper' Vishnyakov,Russia,chopper
1,Spirit,https://www.hltv.org/team/7020/spirit,Dmitry 'sh1ro' Sokolov,Russia,sh1ro
2,Spirit,https://www.hltv.org/team/7020/spirit,Boris 'magixx' Vorobiev,Russia,magixx
3,Spirit,https://www.hltv.org/team/7020/spirit,Myroslav 'zont1x' Plakhotia,Ukraine,zont1x
4,Spirit,https://www.hltv.org/team/7020/spirit,Danil 'donk' Kryshkovets,Russia,donk
5,Vitality,https://www.hltv.org/team/9565/vitality,Dan 'apEX' Madesclaire,France,apEX
6,Vitality,https://www.hltv.org/team/9565/vitality,Robin 'ropz' Kool,Estonia,ropz
7,Vitality,https://www.hltv.org/team/9565/vitality,Mathieu 'ZywOo' Herbaut,France,ZywOo
8,Vitality,https://www.hltv.org/team/9565/vitality,Shahar 'flameZ' Shushan,Israel,flameZ
9,Vitality,https://www.hltv.org/team/9565/vitality,William 'mezii' Merriman,United Kingdom,mezii


In [35]:
from tqdm import tqdm

## GET THE URL FOR ALL THE PLAYER PHOTOS TO MAKE THE API CALLS AND FETCH PNG FILES

# Ensure the photoPlayer column exists
if 'photoPlayer' not in df.columns:
    df['photoPlayer'] = None

unique_pageTeams = df['pageTeam'].unique()

# Initialize the progress bar
progress_bar = tqdm(total=len(unique_pageTeams), desc="Fetching photo URLs")

for pageTeam in unique_pageTeams:
    
    driver = get_chrome_options()
    
    driver.get(pageTeam)
    
    try:
        cookie_accept_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
        )
        cookie_accept_button.click()
    except Exception as e:
        print("Cookies notification not found:", e)
    
    # Wait for the page to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "bodyshot-team-bg"))
    )
    
    # Retrieve the HTML content of the page
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    nameTeam = soup.find('img', class_='team-logo')['alt']
    # Find the player pictures within the team page
    bodyshot_divs = soup.find_all('a', class_='col-custom', limit=5)    
    if len(bodyshot_divs) == 4:
        bodyshot_divs.append(None)
    
    for div in bodyshot_divs:
        if div is None:
            photoPlayer = "https://www.hltv.org/img/static/player/player_silhouette.png"
            nickPlayer = "missing_player"
            namePlayer = "missing_player"
            flagPlayer = "-"
        else:
            photoPlayer = div.find('img', class_='bodyshot-team-img')['src']
            namePlayer = div.find('img', class_='bodyshot-team-img')['title']
            nickPlayer = div.find('img', class_='bodyshot-team-img')['title'].split("'")[1]
            flagPlayer = f"https://www.hltv.org{div.find('img', class_='flag')['src']}"
        
        matching_rows_url = df[(df['nickPlayer'] == nickPlayer) & (df['nameTeam'] == nameTeam)]
        
        # Update the DataFrame with the new player photo URL
        mask = (df['nickPlayer'].str.strip() == nickPlayer.strip()) & (df['nameTeam'].str.strip() == nameTeam.strip())
        df.loc[mask, 'photoPlayer'] = photoPlayer
        df.loc[mask, 'flagPlayer'] = flagPlayer

    driver.quit()
    
    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

df.loc[df['photoPlayer'].isna(), 'namePlayer'] = "Player was Benched"
df.loc[df['photoPlayer'].isna(), 'nickPlayer'] = "Benched"
df.loc[df['photoPlayer'].isna(), 'photoPlayer'] = "https://www.hltv.org/img/static/player/player_silhouette.png"

pd.set_option('display.max_rows', None)
df.to_csv('photoplayers.csv',encoding='utf-8-sig', index=False)


Fetching photo URLs: 100%|██████████| 40/40 [08:31<00:00, 12.78s/it]


In [44]:
display(df)

,nameTeam,pageTeam,namePlayer,countryPlayer,nickPlayer,flagPlayer,photoPlayer,flagPlayer_url
0,Spirit,https://www.hltv.org/team/7020/spirit,Leonid 'chopper' Vishnyakov,Russia,chopper,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/rl_4NtItqpjwLHZL12oMNb.png?ixlib=java-2.1.0&w=400&s=e017b74a48aab673759886475cdfac23,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
1,Spirit,https://www.hltv.org/team/7020/spirit,Dmitry 'sh1ro' Sokolov,Russia,sh1ro,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/-SSnjUN7O5HVENMxG2yReD.png?ixlib=java-2.1.0&w=400&s=3d3dd660e3f0d5287910e6f7d3bbe335,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
2,Spirit,https://www.hltv.org/team/7020/spirit,Boris 'magixx' Vorobiev,Russia,magixx,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/KL_btdQvRC_EPR0c0HdMuZ.png?ixlib=java-2.1.0&w=400&s=c019ad746ce84b793e7ca7d2bcadf57a,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
3,Spirit,https://www.hltv.org/team/7020/spirit,Myroslav 'zont1x' Plakhotia,Ukraine,zont1x,https://www.hltv.org/img/static/flags/30x20/UA.gif,https://img-cdn.hltv.org/playerbodyshot/9W11pewP-V5WZM0KR3GH4v.png?ixlib=java-2.1.0&w=400&s=32961c4bb7501a4fcb91d64f66cf204d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Ukraine.png
4,Spirit,https://www.hltv.org/team/7020/spirit,Danil 'donk' Kryshkovets,Russia,donk,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/snAlUrdIT8ifI4qzNHfLow.png?ixlib=java-2.1.0&w=400&s=e4106a3742a9cd38c16c10e5a642e9f6,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
5,Vitality,https://www.hltv.org/team/9565/vitality,Dan 'apEX' Madesclaire,France,apEX,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/nbkBkY04pLo_1P6b59hVxy.png?ixlib=java-2.1.0&w=400&s=8a0e930e1585977f71c44fabfe3ddac1,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\France.png
6,Vitality,https://www.hltv.org/team/9565/vitality,Robin 'ropz' Kool,Estonia,ropz,https://www.hltv.org/img/static/flags/30x20/EE.gif,https://img-cdn.hltv.org/playerbodyshot/9DBlv6Sx2VqOucU6zCNXcL.png?ixlib=java-2.1.0&w=400&s=daae6f2a89cfea534792b6bfd35c3c72,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Estonia.png
7,Vitality,https://www.hltv.org/team/9565/vitality,Mathieu 'ZywOo' Herbaut,France,ZywOo,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/Xkqvuwl9o12Mi20Vd0lzHl.png?ixlib=java-2.1.0&w=400&s=f64d118affc3f2fbadcebf861d70400d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\France.png
8,Vitality,https://www.hltv.org/team/9565/vitality,Shahar 'flameZ' Shushan,Israel,flameZ,https://www.hltv.org/img/static/flags/30x20/IL.gif,https://img-cdn.hltv.org/playerbodyshot/KDqr5Bwa2JMgtclUOoV3rc.png?ixlib=java-2.1.0&w=400&s=dcf0e087f6987c11084379e92443f416,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Israel.png
9,Vitality,https://www.hltv.org/team/9565/vitality,William 'mezii' Merriman,United Kingdom,mezii,https://www.hltv.org/img/static/flags/30x20/GB.gif,https://img-cdn.hltv.org/playerbodyshot/Xohhl1ddcNtQat_mYmhk-d.png?ixlib=java-2.1.0&w=400&s=63b4c6aca2e8e86c94b8bdbda4bbe60c,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\United Kingdom.png


In [36]:
# SOME DATA CLEANING TO FIX SOME ISSUES
df_backup = df.copy()
display(df)

,nameTeam,pageTeam,namePlayer,countryPlayer,nickPlayer,flagPlayer,photoPlayer,flagPlayer_url
0,Spirit,https://www.hltv.org/team/7020/spirit,Leonid 'chopper' Vishnyakov,Russia,chopper,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/rl_4NtItqpjwLHZL12oMNb.png?ixlib=java-2.1.0&w=400&s=e017b74a48aab673759886475cdfac23,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
1,Spirit,https://www.hltv.org/team/7020/spirit,Dmitry 'sh1ro' Sokolov,Russia,sh1ro,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/-SSnjUN7O5HVENMxG2yReD.png?ixlib=java-2.1.0&w=400&s=3d3dd660e3f0d5287910e6f7d3bbe335,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
2,Spirit,https://www.hltv.org/team/7020/spirit,Boris 'magixx' Vorobiev,Russia,magixx,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/KL_btdQvRC_EPR0c0HdMuZ.png?ixlib=java-2.1.0&w=400&s=c019ad746ce84b793e7ca7d2bcadf57a,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
3,Spirit,https://www.hltv.org/team/7020/spirit,Myroslav 'zont1x' Plakhotia,Ukraine,zont1x,https://www.hltv.org/img/static/flags/30x20/UA.gif,https://img-cdn.hltv.org/playerbodyshot/9W11pewP-V5WZM0KR3GH4v.png?ixlib=java-2.1.0&w=400&s=32961c4bb7501a4fcb91d64f66cf204d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Ukraine.png
4,Spirit,https://www.hltv.org/team/7020/spirit,Danil 'donk' Kryshkovets,Russia,donk,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/snAlUrdIT8ifI4qzNHfLow.png?ixlib=java-2.1.0&w=400&s=e4106a3742a9cd38c16c10e5a642e9f6,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Russia.png
5,Vitality,https://www.hltv.org/team/9565/vitality,Dan 'apEX' Madesclaire,France,apEX,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/nbkBkY04pLo_1P6b59hVxy.png?ixlib=java-2.1.0&w=400&s=8a0e930e1585977f71c44fabfe3ddac1,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\France.png
6,Vitality,https://www.hltv.org/team/9565/vitality,Robin 'ropz' Kool,Estonia,ropz,https://www.hltv.org/img/static/flags/30x20/EE.gif,https://img-cdn.hltv.org/playerbodyshot/9DBlv6Sx2VqOucU6zCNXcL.png?ixlib=java-2.1.0&w=400&s=daae6f2a89cfea534792b6bfd35c3c72,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Estonia.png
7,Vitality,https://www.hltv.org/team/9565/vitality,Mathieu 'ZywOo' Herbaut,France,ZywOo,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/Xkqvuwl9o12Mi20Vd0lzHl.png?ixlib=java-2.1.0&w=400&s=f64d118affc3f2fbadcebf861d70400d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\France.png
8,Vitality,https://www.hltv.org/team/9565/vitality,Shahar 'flameZ' Shushan,Israel,flameZ,https://www.hltv.org/img/static/flags/30x20/IL.gif,https://img-cdn.hltv.org/playerbodyshot/KDqr5Bwa2JMgtclUOoV3rc.png?ixlib=java-2.1.0&w=400&s=dcf0e087f6987c11084379e92443f416,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\Israel.png
9,Vitality,https://www.hltv.org/team/9565/vitality,William 'mezii' Merriman,United Kingdom,mezii,https://www.hltv.org/img/static/flags/30x20/GB.gif,https://img-cdn.hltv.org/playerbodyshot/Xohhl1ddcNtQat_mYmhk-d.png?ixlib=java-2.1.0&w=400&s=63b4c6aca2e8e86c94b8bdbda4bbe60c,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers lags\United Kingdom.png


In [65]:
df = df_backup.copy()

In [66]:
## GET PLAYERS PNGs FROM HLTV WITH TRANSPARENT BACKGROUND, SAVE TO THE \photoPlayers\ FOLDER AND INSERT PNG PATHS INTO A COLUMN IN THE DATAFRAME
import time
headers = {
    "authority": "img-cdn.hltv.org",
    "method": "GET",
    "path": "/playerbodyshot/rl_4NtItqpjwLHZL12oMNb.png?ixlib=java-2.1.0&w=400&s=e017b74a48aab673759886475cdfac23",
    "scheme": "https",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "cookie": "cf_clearance=FV9QsL0R6xwDiTHCGsAJ..NTVDJCiFOzNt7c_SQevY8-1739468146-1.2.1.1-qBWa2RrtZ2csRcGN1qI8f2tf8Xdw8zFEVEAYBSErNskfS3sNvTZ.tBIwif.W6_9IS5VF39zbMVco7NALzmo2LGr0NfC733NpQf_Db7_5E813oDZCGd8R5QLziH8TgGqfzwxEKqEx_8VIk7YZ7Z7_BcSTW8ncRkfkd6ph26IaEbz42D_kYcHi1JTY2jnGuGjrEU5ha1F5F9CrlstCTyhs6smdSKbgsl6JsggRrg9dMFmjh7nQ.JF4yfMxhC22tNh85iE0w.dXcZ_F91u6VxpjaPNQk2ubhgz2z2S.UWChdUM7RfcZfZmLOVGVU9WBH1_O0_bUdPbKDM0Sz2IcKHPsWQ; __cf_bm=YtWAznoBgdWWvBivue5Gy58j2SDoqWJxKjGhAgqYyRw-1739474401-1.0.1.1-vwS3AXNGXgGivAPR0KrZCrzncJN03BjjQYblFWvkkk2MtH.7PzihwzbPKuE_KqLmXCcY67KR5yMHuqcqz7H1gg",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Not(A:Brand\";v=\"99\", \"Google Chrome\";v=\"133\", \"Chromium\";v=\"133\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
}

headers2 = {
    "authority": "www.hltv.org",
    "method": "GET",
    "path": "/img/static/player/player_silhouette.png",
    "scheme": "https",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "cookie": "MatchFilter={%22active%22:false%2C%22live%22:false%2C%22stars%22:1%2C%22lan%22:false%2C%22teams%22:[]}; CookieConsent={stamp:%273NgGg27hj0ejKpNtm87m/YBPSLnL4lNd/yPbJihd+Z9o9RYqFbPN7w==%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27explicit%27%2Cver:1%2Cutc:1733077873420%2Cregion:%27br%27}; statsTablePadding=medium; cf_clearance=P9ijKyCwD8a5rm1I4Boz3YOaqBR15KXAKcree6_8h44-1739476131-1.2.1.1-dpg52IEOA5ltYBkGfLlZQsRgsvPRGYR2AfHk6HJ4m8oAnluwip2cKjL0TN0kf5DaIx8OJv5iz7kaNBoeIts3twQY4_3KEs4XOSHUbZEZWrfA.EjF3OsZeygwVnfJK91sES_y8yPVYsJSP3uz1pgNscIBbG1RZv_UYX.BNabSdLIBlo4uEKe.X6Ch1jm0Im81oOHnlhjfA1MsVywssC_aW9KZJsIBrYi31zz5IVOBb.ujCz1KLpcNEu0Uh3EUKCRuFZO9IzO.ZakFdvmynC9Pi_ltW..OJQEJEvuRZCnuEXiv8oaKkfcR_ZlnO0ZIFE4uM51jyX31Jx4kguz2_vcqZg; __cf_bm=jMzLwRjGt4MV2hgtoWTP8HEeoqiqgocRBhKJuVeVruM-1739476402-1.0.1.1-eFHtIuMZGyiud7rxFO3OxN4WQkKV9Lr5E8NqGkXU4dOV4kqOcEm1i.sjYk9coOrbwvvA5DzlqiZaGUFa.rYffw",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Not(A:Brand\";v=\"99\", \"Google Chrome\";v=\"133\", \"Chromium\";v=\"133\"",
    "sec-ch-ua-arch": "\"x86\"",
    "sec-ch-ua-bitness": "\"64\"",
    "sec-ch-ua-full-version": "\"133.0.6943.99\"",
    "sec-ch-ua-full-version-list": "\"Not(A:Brand\";v=\"99.0.0.0\", \"Google Chrome\";v=\"133.0.6943.99\", \"Chromium\";v=\"133.0.6943.99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-model": "\"\"",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-ch-ua-platform-version": "\"10.0.0\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
}

headers3 = {
    "authority": "www.hltv.org",
    "method": "GET",
    "path": "/img/static/flags/30x20/UY.gif",
    "scheme": "https",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "cookie": "MatchFilter={%22active%22:false%2C%22live%22:false%2C%22stars%22:1%2C%22lan%22:false%2C%22teams%22:[]}; CookieConsent={stamp:%273NgGg27hj0ejKpNtm87m/YBPSLnL4lNd/yPbJihd+Z9o9RYqFbPN7w==%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27explicit%27%2Cver:1%2Cutc:1733077873420%2Cregion:%27br%27}; statsTablePadding=medium; cf_clearance=eMwcm5Xn7zmbnk2m_JFct3sEd.h_Gb9s65S6FNsCsPA-1739493069-1.2.1.1-LuKuIO4cVAaJKiDutA5K0_4l14_6hmWFC6achrycrH.sX1n4lPyck5FA1nR16TqBbgbet.kq4Ik1jyVkgigL3LDJqS2dhQrXxTZl7muRwqiCJDn4H8ZajxIQPKAg3.VckGiyBlEhUIdzUv5xuGJkv_qx0EkOdOuq1RLjVqejb4B82L_LnaLOJ9wWu7Ow2k9p1XieyhmxRBerfg2PLBYUc7ztx_1hA9z3ZLMhBzpITvYxG8vmjFd6J5w4Y7wJFgRJXKF4rC2CzQOb1Rq2NW4g4oCJYyZVJHq1W8Yp9bhrU2fej7bzwOuKcSFVn01ptQyFbNr084YFFpTVBu.5Qlyk6Q; __cf_bm=Zei4mU4WT3we61ryZnV8pBU5TP4hnYNhzuGehlaP_KQ-1739494899-1.0.1.1-rL1JEND5HWPz7eZM_Z2KlzollyOKz3jpuqHHUpRrjri.MIDk1PwRXJgtbcT_8_7oTwHvP9P8f6anDKzJXrD01A",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Not(A:Brand\";v=\"99\", \"Google Chrome\";v=\"133\", \"Chromium\";v=\"133\"",
    "sec-ch-ua-arch": "\"x86\"",
    "sec-ch-ua-bitness": "\"64\"",
    "sec-ch-ua-full-version": "\"133.0.6943.99\"",
    "sec-ch-ua-full-version-list": "\"Not(A:Brand\";v=\"99.0.0.0\", \"Google Chrome\";v=\"133.0.6943.99\", \"Chromium\";v=\"133.0.6943.99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-model": "\"\"",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-ch-ua-platform-version": "\"10.0.0\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
}


# Create directory if it doesn't exist
output_dir = "photoPlayers"
output_dir_flags = os.path.join("photoPlayers", "_flags")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_dir_flags):
    os.makedirs(output_dir_flags)

for index, row in df.iterrows():
    time.sleep(0.1)
    namePlayer = row['namePlayer']
    nickPlayer = row['nickPlayer']
    photoPlayer = row['photoPlayer']
    flagPlayer = row['flagPlayer']
    # countryPlayer = row['countryPlayer']
    
    # response = requests.get(flagPlayer, headers=headers3)
    
    if  "player_silhouette" in photoPlayer:
        response = requests.get('https://www.hltv.org/img/static/player/player_silhouette.png', headers=headers2)
    else:
        response = requests.get(photoPlayer, headers=headers)
    
    
    try:
        img = Image.open(BytesIO(response.content))
        img.save(os.path.join(output_dir, f"{row['nameTeam'].lower().replace(' ', '-')}-{nickPlayer}.png"))
    except UnidentifiedImageError:
        print(f"Could not identify image for URL: {photoPlayer}")
        
    response2 = requests.get(flagPlayer, headers=headers3)
    
    try:
        img = Image.open(BytesIO(response2.content))
        img.save(os.path.join(output_dir_flags, f"{row['countryPlayer'].lower().replace(" ","-")}.png"))
    except UnidentifiedImageError:
        print(f"Could not identify image for URL: {photoPlayer}")
        

df['photoPlayer_url'] = df.apply(lambda row: os.path.abspath(os.path.join(output_dir, f"{row['nameTeam'].lower().replace(' ', '-')}-{row['nickPlayer']}.png")), axis=1)
df['flagPlayer_url']  = df.apply(lambda row: os.path.abspath(os.path.join(output_dir_flags, f"{row['countryPlayer'].lower().replace(' ', '-')}.png")), axis=1)



In [67]:
display(df)

,nameTeam,pageTeam,namePlayer,countryPlayer,nickPlayer,flagPlayer,photoPlayer,flagPlayer_url,photoPlayer_url
0,Spirit,https://www.hltv.org/team/7020/spirit,Leonid 'chopper' Vishnyakov,Russia,chopper,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/rl_4NtItqpjwLHZL12oMNb.png?ixlib=java-2.1.0&w=400&s=e017b74a48aab673759886475cdfac23,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-chopper.png
1,Spirit,https://www.hltv.org/team/7020/spirit,Dmitry 'sh1ro' Sokolov,Russia,sh1ro,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/-SSnjUN7O5HVENMxG2yReD.png?ixlib=java-2.1.0&w=400&s=3d3dd660e3f0d5287910e6f7d3bbe335,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-sh1ro.png
2,Spirit,https://www.hltv.org/team/7020/spirit,Boris 'magixx' Vorobiev,Russia,magixx,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/KL_btdQvRC_EPR0c0HdMuZ.png?ixlib=java-2.1.0&w=400&s=c019ad746ce84b793e7ca7d2bcadf57a,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-magixx.png
3,Spirit,https://www.hltv.org/team/7020/spirit,Myroslav 'zont1x' Plakhotia,Ukraine,zont1x,https://www.hltv.org/img/static/flags/30x20/UA.gif,https://img-cdn.hltv.org/playerbodyshot/9W11pewP-V5WZM0KR3GH4v.png?ixlib=java-2.1.0&w=400&s=32961c4bb7501a4fcb91d64f66cf204d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\ukraine.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-zont1x.png
4,Spirit,https://www.hltv.org/team/7020/spirit,Danil 'donk' Kryshkovets,Russia,donk,https://www.hltv.org/img/static/flags/30x20/RU.gif,https://img-cdn.hltv.org/playerbodyshot/snAlUrdIT8ifI4qzNHfLow.png?ixlib=java-2.1.0&w=400&s=e4106a3742a9cd38c16c10e5a642e9f6,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-donk.png
5,Vitality,https://www.hltv.org/team/9565/vitality,Dan 'apEX' Madesclaire,France,apEX,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/nbkBkY04pLo_1P6b59hVxy.png?ixlib=java-2.1.0&w=400&s=8a0e930e1585977f71c44fabfe3ddac1,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\france.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-apEX.png
6,Vitality,https://www.hltv.org/team/9565/vitality,Robin 'ropz' Kool,Estonia,ropz,https://www.hltv.org/img/static/flags/30x20/EE.gif,https://img-cdn.hltv.org/playerbodyshot/9DBlv6Sx2VqOucU6zCNXcL.png?ixlib=java-2.1.0&w=400&s=daae6f2a89cfea534792b6bfd35c3c72,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\estonia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-ropz.png
7,Vitality,https://www.hltv.org/team/9565/vitality,Mathieu 'ZywOo' Herbaut,France,ZywOo,https://www.hltv.org/img/static/flags/30x20/FR.gif,https://img-cdn.hltv.org/playerbodyshot/Xkqvuwl9o12Mi20Vd0lzHl.png?ixlib=java-2.1.0&w=400&s=f64d118affc3f2fbadcebf861d70400d,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\france.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-ZywOo.png
8,Vitality,https://www.hltv.org/team/9565/vitality,Shahar 'flameZ' Shushan,Israel,flameZ,https://www.hltv.org/img/static/flags/30x20/IL.gif,https://img-cdn.hltv.org/playerbodyshot/KDqr5Bwa2JMgtclUOoV3rc.png?ixlib=java-2.1.0&w=400&s=dcf0e087f6987c11084379e92443f416,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\israel.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-flameZ.png
9,Vitality,https://www.hltv.org/team/9565/vitality,William 'mezii' Merriman,United Kingdom,mezii,https://

In [68]:
# df['photoPlayer_url'] = df.apply(lambda row: os.path.abspath(os.path.join(output_dir, f"{row['nameTeam'].lower().replace(' ', '-')}-{row['nickPlayer']}.png")), axis=1)
# df['flagPlayer_url']  = df.apply(lambda row: os.path.abspath(os.path.join(output_dir_flags, f"{row['countryPlayer']}.png")), axis=1)
df.drop(columns=(['photoPlayer', 'flagPlayer']), inplace=True)
df.rename(columns={'photoPlayer_url': 'photoPlayer', 'flagPlayer_url': 'flagPlayer'}, inplace=True)
df.to_csv('photoplayers.csv',encoding='utf-8-sig', index=False)
display(df)

,nameTeam,pageTeam,namePlayer,countryPlayer,nickPlayer,flagPlayer,photoPlayer
0,Spirit,https://www.hltv.org/team/7020/spirit,Leonid 'chopper' Vishnyakov,Russia,chopper,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-chopper.png
1,Spirit,https://www.hltv.org/team/7020/spirit,Dmitry 'sh1ro' Sokolov,Russia,sh1ro,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-sh1ro.png
2,Spirit,https://www.hltv.org/team/7020/spirit,Boris 'magixx' Vorobiev,Russia,magixx,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-magixx.png
3,Spirit,https://www.hltv.org/team/7020/spirit,Myroslav 'zont1x' Plakhotia,Ukraine,zont1x,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\ukraine.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-zont1x.png
4,Spirit,https://www.hltv.org/team/7020/spirit,Danil 'donk' Kryshkovets,Russia,donk,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\russia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\spirit-donk.png
5,Vitality,https://www.hltv.org/team/9565/vitality,Dan 'apEX' Madesclaire,France,apEX,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\france.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-apEX.png
6,Vitality,https://www.hltv.org/team/9565/vitality,Robin 'ropz' Kool,Estonia,ropz,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\estonia.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-ropz.png
7,Vitality,https://www.hltv.org/team/9565/vitality,Mathieu 'ZywOo' Herbaut,France,ZywOo,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\france.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-ZywOo.png
8,Vitality,https://www.hltv.org/team/9565/vitality,Shahar 'flameZ' Shushan,Israel,flameZ,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\israel.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-flameZ.png
9,Vitality,https://www.hltv.org/team/9565/vitality,William 'mezii' Merriman,United Kingdom,mezii,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\_flags\united-kingdom.png,c:\Users\Thales\Desktop\cs2-hltv-event-scrapper\photoPlayers\vitality-mezii.png


In [58]:
df.to_csv('photoplayers.csv',encoding='utf-8-sig', index=False)